<a href="https://colab.research.google.com/github/damianwgriggs/Perceptual-Grid-Engine-Quantum-Experiment/blob/main/PGE_Quantum_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-ibm-runtime qiskit-aer pylatexenc

import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Parameter
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# ==========================================
# 1. THE PERCEPTUAL GRID LOGIC (ROUND TRIP)
# ==========================================

def create_round_trip_circuit():
    """
    Ping-Pong Teleportation:
    Hop 1: q0 (Start) -> q2 (Grid)
    Hop 2: q2 (Grid)  -> q0 (Home)
    """
    # 3 Qubits, and we need more classical bits for the intermediate steps
    qr = QuantumRegister(3, 'q')

    # Registers for Hop 1 measurements
    crz1 = ClassicalRegister(1, 'crz1')
    crx1 = ClassicalRegister(1, 'crx1')

    # Registers for Hop 2 measurements
    crz2 = ClassicalRegister(1, 'crz2')
    crx2 = ClassicalRegister(1, 'crx2')

    # Final result register
    result_bit = ClassicalRegister(1, 'result')

    qc = QuantumCircuit(qr, crz1, crx1, crz2, crx2, result_bit)

    # --- INITIALIZATION ---
    # Prepare the "NBA Predictor" state on q0 (75% Zero / 25% One)
    qc.rx(np.pi/3, 0)
    qc.barrier()

    # ==========================
    # HOP 1: OUTBOUND (0 -> 2)
    # ==========================

    # 1. Entangle Bridge(q1) and Target(q2)
    qc.h(1)
    qc.cx(1, 2)

    # 2. Bell Measurement on Source(q0) and Bridge(q1)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, crz1)
    qc.measure(1, crx1)

    # 3. Apply Correction to q2
    with qc.if_test((crx1, 1)):
        qc.x(2)
    with qc.if_test((crz1, 1)):
        qc.z(2)

    qc.barrier()

    # ==========================
    # MID-CIRCUIT RESET
    # ==========================
    # We wipe q0 and q1 so we can reuse them.
    # This is a critical feature of your Grid Engine.
    qc.reset(0)
    qc.reset(1)
    qc.barrier()

    # ==========================
    # HOP 2: INBOUND (2 -> 0)
    # ==========================

    # 1. Entangle Bridge(q1) and Home(q0)
    qc.h(1)
    qc.cx(1, 0)

    # 2. Bell Measurement on Grid(q2) and Bridge(q1)
    qc.cx(2, 1)
    qc.h(2)
    qc.measure(2, crz2)
    qc.measure(1, crx2)

    # 3. Apply Correction to Home(q0)
    with qc.if_test((crx2, 1)):
        qc.x(0)
    with qc.if_test((crz2, 1)):
        qc.z(0)

    # ==========================
    # FINAL VERIFICATION
    # ==========================
    # Did the data survive the round trip?
    qc.measure(0, result_bit)

    return qc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.2 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=9faa853fd5e59c5825bf1fc8d4bde5a7a151639a67b3df1117ecdefc23fe1066
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178

In [ ]:
# ==========================================
# 2. STRESS TEST EXECUTION ENGINE
# ==========================================

# --- STEP 0: PASTE YOUR KEY HERE ---
MY_IBM_KEY = "APIKEYHIDDEN"

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Generate the Round Trip circuit
qc = create_round_trip_circuit()
print(f"Circuit Depth: {qc.depth()}")
# Depth will likely be 20-25 now (more stress on the hardware)

# --- RUN ON REAL IBM HARDWARE ---
print("\n[Connecting to IBM Quantum Grid...]")

# 1. Authenticate
QiskitRuntimeService.save_account(channel="ibm_quantum_platform", token=MY_IBM_KEY, overwrite=True)

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Targeting Grid Node: {backend.name}")
print("Status: Transpiling circuit (preparing for atoms)...")

# 2. Transpile
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pm.run(qc)

# 3. Execute
print("Status: Sending STRESS TEST to queue...")
sampler = Sampler(backend)
job = sampler.run([isa_circuit])
print(f"Job ID: {job.job_id()}")

# Wait for result
result = job.result()
print("Status: Round Trip Data received.")

# --- STEP 4: ANALYSIS ---
counts = result[0].data.result.get_counts()

# Logic: The 'result' register is the NEWEST one (Leftmost in the bitstring)
zeros = 0
ones = 0

for key, count in counts.items():
    # Key string format: "Result Hop2_meas Hop1_meas"
    # We only care about the very first character
    final_state = key.split()[0]

    if final_state == '0':
        zeros += count
    else:
        ones += count

total = zeros + ones
percent_zeros = (zeros / total) * 100

print("-" * 30)
print(f"RAW COUNTS: {counts}")
print("-" * 30)
print(f"ROUND TRIP SURVIVAL RATE: {percent_zeros:.1f}%")
print(f"Ideal Target: ~75%")
print("-" * 30)

Circuit Depth: 15

[Connecting to IBM Quantum Grid...]


qiskit_runtime_service.__init__:WARNING:2025-12-13 20:18:02,726: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-13 20:18:02,901: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-13 20:18:04,485: Using instance: open-instance, plan: open


Targeting Grid Node: ibm_fez
Status: Transpiling circuit (preparing for atoms)...
Status: Sending STRESS TEST to queue...
Job ID: d4usjvcgk3fc73auamsg
Status: Round Trip Data received.
------------------------------
RAW COUNTS: {'1': 1151, '0': 2945}
------------------------------
ROUND TRIP SURVIVAL RATE: 71.9%
Ideal Target: ~75%
------------------------------
